<a href="https://colab.research.google.com/github/aaronseymour7/MLQ/blob/main/GFH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install netket
!pip install qiskit

In [ ]:
import netket as nk
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
from netket.experimental.observable import InfidelityOperator

def get_lowest_data(n_sites_list):

    results = []

    for n in n_sites_list:
        print(f"--- Analyzing lowest e_val for N = {n} ---")

        # Lattice and Hamiltonian

        graph=nk.graph.Chain(n)
        hi = nk.hilbert.Spin(s=0.5, N=graph.n_nodes)
        hamiltonian = nk.operator.Heisenberg(hilbert=hi, graph=graph)
        e_exact = None
        ed_result, evecs = nk.exact.lanczos_ed(hamiltonian, k=1, compute_eigenvectors=True)
        e_exact = float(ed_result[0])
        target_vec = evecs[:, 0]
        lmodel = nk.models.LogStateVector(hi)
        l_vstate = nk.vqs.FullSumState(hi, lmodel)
        log_psi = jnp.log(target_vec.astype(complex))

        l_vstate.parameters = {"logstate": log_psi}


        #print(f"Lanczos (exact): {e_exact}")
        #Neural Network (RBM)
        model = nk.models.RBM(alpha=2, param_dtype=complex) #RBM DENSITY COEFFICIENT
        sampler = nk.sampler.MetropolisLocal(hi) #using Metropolis Exchange bc total_sz = 0

        vstate = nk.vqs.MCState(sampler, model, n_samples=1008)

        '''
        # Neel state

        n_chains = vstate.sampler.n_chains
        neel = np.ones(n)
        neel[1::2] = -1
        neel_samples = np.tile(neel, (n_chains, 1)).astype(np.int8)
        vstate.sampler_state = vstate.sampler_state.replace(σ=jnp.array(neel_samples))
        '''

        # Optimizer: Stochastic Reconfiguration (SR)
        optimizer = nk.optimizer.Sgd(learning_rate=0.01)  #LEARNING RATE
        #sr = nk.optimizer.SR(diag_shift=0.1)
        #vmc = nk.driver.VMC(hamiltonian, optimizer, variational_state=vstate, preconditioner=sr)
        vmc = nk.driver.VMC_SR(hamiltonian, optimizer, diag_shift=0.01, variational_state=vstate)
        # Run Optimization
        vmc.run(n_iter=500)
        energy_stats = vstate.expect(hamiltonian)
        e_vmc = float(energy_stats.mean.real)
        e_error = float(energy_stats.error_of_mean)
        overlap, o_error = get_overlap(l_vstate, vstate)
        results.append({
            "exact": l_vstate,
            "VMC_vstate": vstate,
            "N": n,
            "VMC": e_vmc,
            "Lanczos": e_exact,
            "diff": abs(e_vmc - e_exact),
            "VMC_err": e_error,
            "overlap": overlap
        })

    return results

def get_highest_data(n_sites_list):

    results = []

    for n in n_sites_list:
        print(f"--- Analyzing Highest e_val for N = {n} ---")

        # Lattice and Hamiltonian
        graph=nk.graph.Chain(n)
        hi = nk.hilbert.Spin(s=0.5, N=graph.n_nodes)
        hamiltonian = -1 * nk.operator.Heisenberg(hilbert=hi, graph=graph)

        e_exact = None
        ed_result, evecs = nk.exact.lanczos_ed(hamiltonian, k=1,compute_eigenvectors=True)
        e_exact = -1 * float(ed_result[0])
        target_vec = evecs[:, 0]
        lmodel = nk.models.LogStateVector(hi)
        l_vstate = nk.vqs.FullSumState(hi, lmodel)
        log_psi = jnp.log(target_vec.astype(complex))

        l_vstate.parameters = {"logstate": log_psi}

        #Neural Network (RBM)
        model = nk.models.RBM(alpha=2, param_dtype=complex) #RBM DENSITY COEFFICIENT
        sampler = nk.sampler.MetropolisLocal(hi)  #using Metropolis Local sampler instead of exchange bc total_sz != 0
        vstate = nk.vqs.MCState(sampler, model, n_samples=1008)

        # Optimizer: Stochastic Reconfiguration (SR)
        optimizer = nk.optimizer.Sgd(learning_rate=0.01) #LEARNING RATE
        sr = nk.optimizer.SR(diag_shift=0.1)
        vmc = nk.driver.VMC_SR(hamiltonian, optimizer, diag_shift=0.01, variational_state=vstate)
        # Run Optimization
        vmc.run(n_iter=500)
        energy_stats = vstate.expect(hamiltonian)
        e_vmc = -1*float(energy_stats.mean.real)
        e_error = float(energy_stats.error_of_mean)
        overlap, o_error = get_overlap(l_vstate, vstate)
        results.append({
            "exact_vstate": l_vstate,
            "VMC_vstate" : vstate,
            "N": n,
            "VMC": e_vmc,
            "Lanczos": e_exact,
            "diff": abs(e_vmc - e_exact),
            "VMC_err": e_error,
            "overlap": overlap,
        })

    return results
def get_1es_data(n_sites_list):

    results = []

    for n in n_sites_list:

        print(f"--- Analyzing First Excited State for N = {n} ---")

        # Lattice and Hamiltonian
        target_sz = 1 if n % 2 == 0 else 1.5
        graph=nk.graph.Chain(n)
        hi = nk.hilbert.Spin(s=0.5,total_sz=target_sz, N=graph.n_nodes)
        lanczos_hi = nk.hilbert.Spin(s=0.5, N=graph.n_nodes)
        hamiltonian = nk.operator.Heisenberg(hilbert=hi, graph=graph)
        lanczos_ham = nk.operator.Heisenberg(hilbert=hi, graph=graph)
        e_exact = None
        ed_result, evecs = nk.exact.lanczos_ed(lanczos_ham, k=1, compute_eigenvectors=True)
        e_exact = float(ed_result[0])
        target_vec = evecs[:, 0]
        lmodel = nk.models.LogStateVector(hi)
        l_vstate = nk.vqs.FullSumState(hi, lmodel)
        log_psi = jnp.log(target_vec.astype(complex))

        l_vstate.parameters = {"logstate": log_psi}

        #Neural Network (RBM)
        model = nk.models.RBM(alpha=2, param_dtype=complex) #RBM DENSITY COEFFICIENT
        sampler = nk.sampler.MetropolisExchange(hi, graph=graph) #using Metropolis Exchange bc total_sz = 0

        vstate = nk.vqs.MCState(sampler, model, n_samples=1008)

        '''
        # modified Neel state

        n_chains = vstate.sampler.n_chains
        neel = np.ones(n)
        neel[2::2] = -1
        neel_samples = np.tile(neel, (n_chains, 1)).astype(np.int8)
        #print(f"Initial magnetization sum: {jnp.sum(neel_samples[0])}") #total magnetization check
        #vstate.sampler_state = vstate.sampler_state.replace(σ=jnp.array(neel_samples))
        '''

        # Optimizer: Stochastic Reconfiguration (SR)
        optimizer = nk.optimizer.Sgd(learning_rate=0.01)  #LEARNING RATE
        #sr = nk.optimizer.SR(diag_shift=0.1)
        #vmc = nk.driver.VMC(hamiltonian, optimizer, variational_state=vstate, preconditioner=sr)
        vmc = nk.driver.VMC_SR(hamiltonian, optimizer, diag_shift=0.01, variational_state=vstate)
        # Run Optimization
        vmc.run(n_iter=500)
        energy_stats = vstate.expect(hamiltonian)
        e_vmc = float(energy_stats.mean.real)
        e_error = float(energy_stats.error_of_mean)
        overlap, o_error = get_overlap(l_vstate, vstate)
        results.append({
            "exact_vstate": l_vstate,
            "VMC_vstate" : vstate,
            "N": n,
            "VMC": e_vmc,
            "Lanczos": e_exact,
            "diff": abs(e_vmc - e_exact),
            "VMC_err": e_error,
            "overlap": overlap,
        })
    return results


def printer(data):
  for d in data:
    status = f"Diff: {d['diff']:.8f}" if d['diff'] is not None else "Exact not computed"
    print(f"N={d['N']} | VMC: {d['VMC']:.8f} | Lanczos: {d['Lanczos']:.8f} | Abs Difference: {d['diff']:.8f} | Overlap: {d['overlap']} \n")


def plot_simulation_results(results_list):

    low_data = [results_list[i][0] for i in range(0, len(results_list), 3)]
    es_data  = [results_list[i][0] for i in range(1, len(results_list), 3)]
    hi_data  = [results_list[i][0] for i in range(2, len(results_list), 3)]

    datasets = [low_data, es_data, hi_data]
    titles = ['Ground State', 'First Excited', 'Highest']
    colors = ['red', 'blue', 'green']

    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    for i, ax in enumerate(axes):
        data = datasets[i]
        ns = [d['N'] for d in data]
        diffs = [d['diff'] for d in data]
        errors = [d['VMC_err'] for d in data]

        ax.errorbar(ns, diffs, yerr=errors, fmt='-o', color=colors[i],
                    capsize=6, elinewidth=2, markersize=8, label='VMC Stat. Error')

        ax.set_title(titles[i], fontsize=14, fontweight='bold')
        ax.set_xlabel('System Size (N)', fontsize=12)
        ax.set_ylabel('Abs Difference |VMC - Lanczos|', fontsize=12)
        ax.set_xticks(ns)
        ax.grid(True, linestyle='--', alpha=0.6)
        ax.legend()

    plt.suptitle("VMC Accuracy (1D Heisenberg)", fontsize=16, y=1.05)
    plt.tight_layout()
    plt.show()

def get_all_data(n_sizes):
  results = []
  for n in n_sizes:

    data = get_lowest_data([n])
    results.append(data)
    printer(data)
    data = get_1es_data([n])
    results.append(data)
    printer(data)
    data = get_highest_data([n])
    results.append(data)
    printer(data)
  plot_simulation_results(results)

def get_overlap(vstate, target):
  inf_op = InfidelityOperator(target)
  result = vstate.expect(inf_op)
  infidelity_mean = result.mean
  error = result.error_of_mean
  fidelity = 1 - infidelity_mean
  return fidelity, error






n_sizes = [6,7,8]
#n_sizes = [6]
get_all_data(n_sizes)


